<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/ensemble_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

df_1 = pd.read_csv('/content/submission_ANN_Fold_0.csv',index_col=0)
df_2 = pd.read_csv('/content/submission_ANN_Fold_1.csv',index_col=0)
df_3 = pd.read_csv('/content/submission_ANN_Fold_2.csv',index_col=0)
df_4 = pd.read_csv('/content/submission_ANN_Fold_3.csv',index_col=0)
df_5 = pd.read_csv('/content/submission_ANN_Fold_4.csv',index_col=0)

In [17]:
final = pd.DataFrame()

final['txkey'] = df_1['txkey']

final['pred_1'] = df_1['fraud_ind']
final['pred_2'] = df_2['fraud_ind']
final['pred_3'] = df_3['fraud_ind']
final['pred_4'] = df_4['fraud_ind']
final['pred_5'] = df_5['fraud_ind']

final['fraud_ind'] = final.drop('txkey',axis=1).mean(axis=1)
final = final[['txkey','fraud_ind']]

final.head()

,txkey,fraud_ind
1521787,592489,0.0
1521788,592452,0.0
1521789,590212,0.0
1521790,590209,0.0
1521791,592488,0.0


In [18]:
final['fraud_ind'].unique()

array([0. , 0.2, 0.4, 0.6, 1. , 0.8])

In [0]:
def f(x):
  if x>=0.8:
    return 1
  else:
    return 0
final['fraud_ind'] = final['fraud_ind'].apply(f)

In [20]:
ps={}

for index,data in enumerate([df_1,df_2,df_3,df_4,df_5,final]):
  ps['p'+str(index)] = round((data['fraud_ind'].value_counts()[1] / #盜刷樣本數
                len(data['fraud_ind']))*100 ,5) #總數

pd.DataFrame(ps,index=['盜刷比例%']).rename(columns={'p5':'final_p'})

,p0,p1,p2,p3,p4,final_p
盜刷比例%,1.31811,1.2804,1.27376,1.274,1.35297,0.89479


# 保存預測結果

In [0]:
final.to_csv("./submission_ANN_Voting_big_than_0_4_result.csv")

In [21]:
print(final["fraud_ind"].value_counts())

0    417892
1      3773
Name: fraud_ind, dtype: int64
